In [1]:
from utils import *

In [ ]:
def rgb2yiq(video):

    yiq_from_rgb = np.array([[0.299, 0.587, 0.114],
                             [0.596, -0.274, -0.322],
                             [0.211, -0.523, 0.312]])
    t = np.dot(video, yiq_from_rgb.T).astype('uint8')
    return t


def yiq2rgb(video):
    
    rgb_from_yiq = np.array([[1, 0.956, 0.621],
                             [1, -0.272, -0.647],
                             [1, -1.106, 1.703]])
    t = np.dot(video, rgb_from_yiq.T).astype('uint8')
    return t


# PROCESAMIENTO DE "baby2.mp4" EN RGB

In [2]:
baby2 = './data/baby2.mp4'

#Separo el video en frames
frames_baby2, nro_Frames_baby2, fps_baby2, filas_baby2, columnas_baby2 = video_to_frames(baby2)

#Video gaussiano
video_gaussiano_baby2 = gaussian_video(frames = frames_baby2, levels = 6)

#Filtrado temporal
amplified_frames_baby2 = ideal_filter(frames_baby2, video_gaussiano_baby2, chromAttenuation = 1, 
                                      fl = 140/40, fh = 160/40, frame_rate = fps_baby2, alpha = 150)

frames_to_video(amplified_frames_baby2, 'Baby2_amplified_RGB', fps_baby2)

Nos da peor que el resultado en RGB

# PROCESAMIENTO DE "face.mp4" EN RGB

In [ ]:
face = './data/face.mp4'

#Separo el video en frames
frames_face, nro_Frames_face, fps_face, filas_face, columnas_face = video_to_frames(face)

#Video gaussiano
video_gaussiano_face = gaussian_video(frames = frames_face, levels = 4)

#Filtrado temporal
amplified_frames_face = ideal_filter(frames_face, video_gaussiano_face, chromAttenuation = 1, 
                                      fl = 50/60, fh = 60/60, frame_rate = fps_face, alpha = 50)

frames_to_video(amplified_frames_face, 'face_amplified_RGB', fps_face)

# PROCESAMIENTO DE "graci.mp4" EN RGB

In [ ]:
graci = './data/graci.mp4'

#Separo el video en frames
frames_graci, nro_Frames_graci, fps_graci, filas_graci, columnas_graci = video_to_frames(graci)

#Video gaussiano
video_gaussiano_graci = gaussian_video(frames = frames_graci, levels = 6)

#Filtrado temporal
amplified_frames_graci = ideal_filter(frames_graci, video_gaussiano_graci, chromAttenuation = 1, 
                                      fl = 50/60, fh = 60/60, frame_rate = fps_graci, alpha = 150)

frames_to_video(amplified_frames_graci, 'graci_amplified_RGB', fps_graci)

# PROCESAMIENTO DE "face2.mp4" EN RGB

In [ ]:
face2 = './data/face2.mp4'

#Separo el video en frames
frames_face2, nro_Frames_face2, fps_face2, filas_face2, columnas_face2 = video_to_frames(face2)

#Video gaussiano
video_gaussiano_face2 = gaussian_video(frames = frames_face2, levels = 6)

#Filtrado temporal
amplified_frames_face2 = ideal_filter(frames_face2, video_gaussiano_face2, chromAttenuation = 1, 
                                      fl = 50/60, fh = 60/60, frame_rate = fps_face2, alpha = 150)

frames_to_video(amplified_frames_face2, 'face2_amplified_RGB', fps_face2)

# PROCESAMIENTO DE "andres.mp4" EN RGB

In [ ]:
andres = './data/andres.mp4'

#Separo el video en frames
frames_andres, nro_Frames_andres, fps_andres, filas_andres, columnas_andres = video_to_frames(andres)

#Video gaussiano
video_gaussiano_andres = gaussian_video(frames = frames_andres, levels = 6)

#Filtrado temporal
amplified_frames_andres = ideal_filter(frames_andres, video_gaussiano_andres, chromAttenuation = 1, 
                                      fl = 50/60, fh = 60/60, frame_rate = fps_andres, alpha = 150)

frames_to_video(amplified_frames_andres, 'andres_amplified_RGB', fps_andres)

# PROCESAMIENTO DE "baby2.mp4" EN YIQ

In [ ]:
baby2 = './data/baby2.mp4'

frames_baby2, nro_Frames, fps, filas, columnas = video_to_frames(baby2)

frames_baby2_yiq = []
for i in range(nro_Frames):
    frames_baby2_yiq.append(rgb2yiq(frames_baby2[i]))
    
video_gaussiano_baby2_yiq = gaussian_video(frames = frames_baby2_yiq, levels = 6)

amplified_frames_baby2_yiq = ideal_filter(frames_baby2_yiq, video_gaussiano_baby2_yiq, chromAttenuation = 1, 
                                      fl = 140/40, fh = 160/40, frame_rate = fps, alpha = 150)

amplified_frames_baby2_rgb = []
for i in range(nro_Frames):
    amplified_frames_baby2_rgb.append(yiq2rgb(amplified_frames_baby2_yiq[i]))
    
frames_to_video(amplified_frames_baby2_rgb, 'Baby2_amplified_YIQ', fps)

In [ ]:
def butter_filter_and_collapse_lpyr(original_frames, laplacian_video_pyramid, alpha, 
                                    spatial_cutoff, fl, fh, levels, frame_rate, columnas, filas):

    '''
    Función que aplica el filtro 
    
        frames_originales: lista de frames originales del video (salida de video_to_frames)
        laplacian_video_pyramid: pirámide laplaciana a filtrar (salida de la función frames_to_laplacian_pyr)
        levels: nro de niveles de la piramide.
        alpha: factor de amplificación
        spatial_cutoff: valor de corte de frecuencia espacial
        fl: frecuencia de corte baja
        fh: frecuencia de corte alta
        frame_rate: frames por segundo (salida de video_to_frames)
        columnas: columnas de un frame del video (salida de la función video_to_frames)
        filas: filas de un frame del video (salida de la función video_to_frames)
    
    '''

    # apply butterworth filter
    amplified_frames = apply_butter_filter(laplacian_video_pyramid, levels, alpha, spatial_cutoff, 
                             fl, fh, frame_rate, columnas, filas)

    reconstructed_frames = collapse_lpyr(amplified_frames, levels)

#     # chromatic attenuation
#     reconstructed_frames[:][:][:][1] *= chromAttenuation
#     reconstructed_frames[:][:][:][2] *= chromAttenuation

    # Add to original
    reconstructed_frames += original_frames

#     # Cutoff wrong values
#     reconstructed_frames[reconstructed_frames < 0] = 0
#     reconstructed_frames[reconstructed_frames > 255] = 255

    return recon_frames 


In [ ]:
# Calculo niveles de face2
levels_face2 = laplacian_pyr_levels(columnas_face2, filas_face2)

# Armo pirámide laplaciana
lpyr_face2 = frames_to_laplacian_pyr(frames_face2, levels_face2)

In [ ]:
#Aplico filtro y reconstruyo piramide
frames_amplificados_reconstruidos = butter_filter_and_collapse_lpyr(frames_face2, lpyr_face2, alpha = 20, 
                                    spatial_cutoff = 80, fl = 0.5, fh = 10, levels = levels_face2, 
                                frame_rate = fps_face2, columnas = columnas_face2, filas = filas_face2)